In [1]:
import numpy as np
import pandas as pd
import mlflow

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
import lightgbm as lgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sudhirjoon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
mlflow.set_tracking_uri("http://13.60.79.0:5000")
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://capstone-yt-mlflow-bucket/542391321365143307', creation_time=1735485482149, experiment_id='542391321365143307', last_update_time=1735485482149, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [3]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
display(df.head(2))
display(df.shape)

,clean_comment,category
0,family mormon never tried explain still stare ...,1
1,buddhism much lot compatible christianity espe...,1


(36662, 2)

In [4]:
# Define TF-IDF vectorization parameters
ngram_range = (1, 2)  # Bigram setting
max_features = 1000  # Set max_features to 1000 for TF-IDF

# Train-test split before vectorization and resampling
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category']
)

# Vectorization using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X_train_vec = vectorizer.fit_transform(X_train)  # Fit on training data
X_test_vec = vectorizer.transform(X_test)  # Transform test data

# Handle imbalance using ADASYN
adasyn = ADASYN(random_state=42)
X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)


# Function to log model and metrics in MLFlow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model name and experiment type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log model algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model and evaluate
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log detailed classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model itself
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Optuna objective function for LightGBM
def objective_lgbm(trial):
    # Suggest hyperparameters for LGBM
    num_leaves = trial.suggest_int("num_leaves", 10, 150)  # Number of leaves in one tree
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 0.1)  # Learning rate
    n_estimators = trial.suggest_int("n_estimators", 50, 500)  # Number of boosting iterations
    max_depth = trial.suggest_int("max_depth", 3, 30)  # Maximum depth of tree

    # Initialize the LGBMClassifier
    model = lgb.LGBMClassifier(
        num_leaves=num_leaves,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Train and evaluate the model
    model.fit(X_train_vec, y_train)
    y_pred = model.predict(X_test_vec)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Function to log model and metrics in MLFlow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model name and experiment type
        mlflow.set_tag("mlflow.runName", f"{model_name}_ADASYN_TFIDF_Bigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log model algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model and evaluate
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log detailed classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model itself
        mlflow.sklearn.log_model(model, f"{model_name}_model")

# Run Optuna to optimize LightGBM hyperparameters
def run_optuna_experiment():
    # Create an Optuna study
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lgbm, n_trials=50)

    # Get the best hyperparameters and retrain the best model
    best_params = study.best_params
    best_model = lgb.LGBMClassifier(
        num_leaves=best_params['num_leaves'],
        learning_rate=best_params['learning_rate'],
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        random_state=42
    )

    # Log the best model with MLFlow
    log_mlflow("LGBM", best_model, X_train_vec, X_test_vec, y_train, y_test)

# Run the Optuna experiment for LGBM
run_optuna_experiment()

[I 2024-12-30 11:31:26,786] A new study created in memory with name: no-name-25e81a66-73f7-44e2-af27-7c5f98199364
/var/folders/yj/2fr85zvn2sbbpv3jsq4y3kwh0000gn/T/ipykernel_50194/1187553769.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 0.1)  # Learning rate


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.080683 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100027
[LightGBM] [Info] Number of data points in the train set: 35868, number of used features: 991
[LightGBM] [Info] Start training from score -1.071711
[LightGBM] [Info] Start training from score -1.184681
[LightGBM] [Info] Start training from score -1.044880


[I 2024-12-30 11:31:49,861] Trial 0 finished with value: 0.6838947224873858 and parameters: {'num_leaves': 78, 'learning_rate': 0.0024060157986524272, 'n_estimators': 231, 'max_depth': 17}. Best is trial 0 with value: 0.6838947224873858.
/var/folders/yj/2fr85zvn2sbbpv3jsq4y3kwh0000gn/T/ipykernel_50194/1187553769.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 0.1)  # Learning rate


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.105911 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100027
[LightGBM] [Info] Number of data points in the train set: 35868, number of used features: 991
[LightGBM] [Info] Start training from score -1.071711
[LightGBM] [Info] Start training from score -1.184681
[LightGBM] [Info] Start training from score -1.044880


[I 2024-12-30 11:31:54,396] Trial 1 finished with value: 0.4301104595663439 and parameters: {'num_leaves': 14, 'learning_rate': 1.055581250532087e-05, 'n_estimators': 126, 'max_depth': 14}. Best is trial 0 with value: 0.6838947224873858.
/var/folders/yj/2fr85zvn2sbbpv3jsq4y3kwh0000gn/T/ipykernel_50194/1187553769.py:53: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 0.1)  # Learning rate


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.087916 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 100027
[LightGBM] [Info] Number of data points in the train set: 35868, number of used features: 991
[LightGBM] [Info] Start training from score -1.071711
[LightGBM] [Info] Start training from score -1.184681
[LightGBM] [Info] Start training from score -1.044880


[I 2024-12-30 12:21:58,456] Trial 49 finished with value: 0.7951725078412655 and parameters: {'num_leaves': 10, 'learning_rate': 0.09832002186955217, 'n_estimators': 450, 'max_depth': 9}. Best is trial 48 with value: 0.797490795036138.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.078662 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 100027
[LightGBM] [Info] Number of data points in the train set: 35868, number of used features: 991
[LightGBM] [Info] Start training from score -1.071711
[LightGBM] [Info] Start training from score -1.184681
[LightGBM] [Info] Start training from score -1.044880


2024/12/30 12:22:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run LGBM_ADASYN_TFIDF_Bigrams at: http://13.60.79.0:5000/#/experiments/542391321365143307/runs/26a0841c651c4c9292db54030dd26cc8
🧪 View experiment at: http://13.60.79.0:5000/#/experiments/542391321365143307
